In [7]:
import pandas as PD
import numpy as NP
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
from selenium.common.exceptions import TimeoutException  
# import ActionChains mouse operation  
from selenium.webdriver.common.action_chains import ActionChains 

from selenium.webdriver.chrome.options import Options
import time
from lxml import etree


In [8]:
def string_cleaner(source):
    string = '' 
    
    for char in source:
        if 0 <= ord(char) <= 126:
            string += char
    return string

# Dynamic get html

In [9]:
def get_html(url):
    try:
        chrome_options=Options()
        chrome_options.add_argument('--headless')
        driver= webdriver.Chrome(chrome_options=chrome_options)  
        driver.get(url)
        temp_height=0
 
        while True:
            #循环将滚动条下拉
            driver.execute_script("window.scrollBy(0,100000)")
            #sleep一下让滚动条反应一下
            time.sleep(5)
            #获取当前滚动条距离顶部的距离
            check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            #如果两者相等说明到底了
            if check_height==temp_height:
                break
            temp_height=check_height
            print('*****Processing temp_height = {temp_height}*****'.format(temp_height=temp_height))
        print('done')
        page = driver.page_source
        source = page.encode("utf8")
        soup_list = BeautifulSoup(source, 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")
        return soup_list
    except:
        return "ERROR"

In [10]:
def get_green_house_url(url):
    try:
        response  = requests.get(url)
        response.encoding="utf-8"
        source = response.text
        content = BeautifulSoup(source, 'lxml').body.find('div', id="content").get_text()
        return content
    except:
        return "ERROR"

In [11]:
# description = get_green_house_url('https://boards.greenhouse.io/mythicalgames/jobs/5463953003')
# description

In [12]:
soup_list = get_html("https://careers.fenbushicapital.vc/jobs")

/tmp/ipykernel_24096/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1378*****
*****Processing temp_height = 4514*****
*****Processing temp_height = 7639*****
*****Processing temp_height = 10795*****
*****Processing temp_height = 13804*****
*****Processing temp_height = 16911*****
*****Processing temp_height = 20211*****
*****Processing temp_height = 23321*****
*****Processing temp_height = 26429*****
*****Processing temp_height = 29553*****
*****Processing temp_height = 32748*****
*****Processing temp_height = 35862*****
*****Processing temp_height = 39098*****
*****Processing temp_height = 42370*****
*****Processing temp_height = 45449*****
*****Processing temp_height = 48623*****
*****Processing temp_height = 51835*****
*****Processing temp_height = 55040*****
*****Processing temp_height = 58221*****
*****Processing temp_height = 61349*****
*****Processing temp_height = 64491*****
*****Processing temp_height = 67692*****
*****Processing temp_height = 70859*****
*****Processing temp_height = 73934*****
*****Processing tem

In [13]:
# soup_list

In [14]:
# get the initializd infomation
tag_list = []

job_title_list = []
type_list = {}
industry_list = {}
contract_list = {}
location_list = {}
equity_list = {}
description_list = {}
salary_list = {}
label_list = {}
achievements_list = {}
# loop = 0

for tag in soup_list.find_all('div',class_="job-list-job"):
        # print(tag)
        company_name = tag.find('a', class_ = "job-list-job-company-link").text
        job_name = tag.find('a', target = "_blank").text
        job_title = company_name + ' + ' + job_name
        raw_data = tag.find('a', target = "_blank")
        job_title_list.append(job_title)
        type_list[job_title] = 'Full-time'
        industry_list[job_title] = ['Others']
        contract_list[job_title] = 'talentre'
        # location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-badge-text").text if (not tag.find('span', class_ = "job-list-badge-text").text[0:6] == "Posted") else "Not Posted"
        location_list[job_title] = string_cleaner(tag.find('span', class_ = "job-list-company-meta-item").text)
        equity_list[job_title] = "Not Posted"
        description_list[job_title] = raw_data['href']
        salary_list[job_title] = "Salary Negotiable"
        label_list[job_title] = []
        achievements_list[job_title] = 'Null'
        for tag_ in tag.find_all('span', class_ = 'job-list-job-skill job-list-job-skill-clickable'):
                label_list[job_title].append(string_cleaner(tag_.text))
        # loop += 1
        # if(loop == 10):
        #     break
        # for tag_ in tag.find_all('span',attrs={ 'data-toggle': 'tooltip'}):
        #     if('$' in tag_.text):
        #         tmp = tag_.text.split('$')[1].replace(',','').replace(')','')

In [15]:
#adjust the description_list
for single in job_title_list:
    if ('https://boards.greenhouse.io/' in description_list[single]):
        content = string_cleaner(get_green_house_url(description_list[single]))
        if content == "ERROR":
            continue
        else:
            description_list[single] = content
        # print('YES')

In [16]:
# adjust the type_list
soup_list_internship = get_html("https://careers.fenbushicapital.vc/jobs?internshipOnly=true")
for tag in soup_list_internship.find_all('div',class_="job-list-job"):
    company_name = tag.find('a', class_ = "job-list-job-company-link").text
    job_name = tag.find('a', target = "_blank").text
    job_title = company_name + ' + ' + job_name
    type_list[job_title] = 'Internship'

/tmp/ipykernel_24096/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4562*****
*****Processing temp_height = 6556*****
done


In [17]:
# adjust the type_list
url_list = {f"https://careers.fenbushicapital.vc/jobs?jobTypes=Engineer":"Engineering",
            f"https://careers.fenbushicapital.vc/jobs?markets=FinTech": "Finance",
            f"https://careers.fenbushicapital.vc/jobs?markets=Financial+Services": "Finance",
            f"https://careers.fenbushicapital.vc/jobs?jobTypes=Marketing": "Marketing",
            f"https://careers.fenbushicapital.vc/jobs?markets=Investment%2FTrading": "Non-Tech",
            f"https://careers.fenbushicapital.vc/jobs?markets=Custody": "Customer Support",
            f"https://careers.fenbushicapital.vc/jobs?markets=Infrastructure": "Product",
            f"https://careers.fenbushicapital.vc/jobs?skills=User+Experience+Design": "Design",
            f"https://careers.fenbushicapital.vc/jobs?markets=Stablecoin": "Operations",
            }

In [18]:
for url in url_list.keys():
    print("********************Processing {url} for label {label}********************".format(url = url, label = url_list[url]))
    soup_list_type = get_html(url)
    for tag in soup_list_type.find_all('div',class_="job-list-job"):
        company_name = tag.find('a', class_ = "job-list-job-company-link").text
        job_name = tag.find('a', target = "_blank").text
        job_title = company_name + ' + ' + job_name
        if(job_title in industry_list.keys()):
            if (industry_list[job_title] == ['Others']):
                industry_list[job_title] = [url_list[url]]
            elif (not url_list[url] in industry_list[job_title]):
                industry_list[job_title].append(url_list[url])
            else: 
                continue
        else:
            continue


********************Processing https://careers.fenbushicapital.vc/jobs?jobTypes=Engineer for label Engineering********************


/tmp/ipykernel_24096/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4604*****
*****Processing temp_height = 7575*****
*****Processing temp_height = 10797*****
*****Processing temp_height = 13929*****
*****Processing temp_height = 17190*****
*****Processing temp_height = 20300*****
*****Processing temp_height = 23509*****
*****Processing temp_height = 26669*****
*****Processing temp_height = 29744*****
*****Processing temp_height = 32960*****
*****Processing temp_height = 36140*****
*****Processing temp_height = 39237*****
*****Processing temp_height = 42491*****
*****Processing temp_height = 45644*****
*****Processing temp_height = 48775*****
*****Processing temp_height = 51928*****
*****Processing temp_height = 55059*****
*****Processing temp_height = 58240*****
*****Processing temp_height = 61407*****
*****Processing temp_height = 64650*****
*****Processing temp_height = 67813*****
*****Processing temp_height = 70948*****
*****Processing temp_height = 74111*****
*****Processing temp_height = 77320*****
*****Processing te

/tmp/ipykernel_24096/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 1289*****
done
********************Processing https://careers.fenbushicapital.vc/jobs?markets=Financial+Services for label Finance********************


...

*****Processing temp_height = 4415*****
*****Processing temp_height = 7491*****
*****Processing temp_height = 10482*****
*****Processing temp_height = 13656*****
*****Processing temp_height = 16759*****
*****Processing temp_height = 19747*****
*****Processing temp_height = 22781*****
*****Processing temp_height = 25759*****
*****Processing temp_height = 28855*****
*****Processing temp_height = 31938*****
*****Processing temp_height = 34733*****
*****Processing temp_height = 37564*****
*****Processing temp_height = 40300*****
*****Processing temp_height = 43089*****
*****Processing temp_height = 45814*****
*****Processing temp_height = 47208*****
done
********************Processing https://careers.fenbushicapital.vc/jobs?skills=User+Experience+Design for label Design********************


/tmp/ipykernel_24096/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4446*****
*****Processing temp_height = 6033*****
done
********************Processing https://careers.fenbushicapital.vc/jobs?markets=Stablecoin for label Operations********************


/tmp/ipykernel_24096/375155834.py:5: DeprecationWarning: use options instead of chrome_options
  driver= webdriver.Chrome(chrome_options=chrome_options)


*****Processing temp_height = 4672*****
*****Processing temp_height = 8098*****
*****Processing temp_height = 11331*****
*****Processing temp_height = 14655*****
*****Processing temp_height = 17846*****
*****Processing temp_height = 21110*****
*****Processing temp_height = 24410*****
*****Processing temp_height = 27594*****
*****Processing temp_height = 30866*****
*****Processing temp_height = 34155*****
*****Processing temp_height = 37472*****
*****Processing temp_height = 40733*****
*****Processing temp_height = 44029*****
*****Processing temp_height = 47248*****
*****Processing temp_height = 50548*****
*****Processing temp_height = 53823*****
*****Processing temp_height = 57094*****
*****Processing temp_height = 60415*****
*****Processing temp_height = 63267*****
*****Processing temp_height = 66139*****
*****Processing temp_height = 69033*****
*****Processing temp_height = 71926*****
*****Processing temp_height = 74830*****
*****Processing temp_height = 76845*****
done


In [19]:
# show all the results
count = 0
total_columns = ['Job title', 'Type of position', 'Job Location', 'Job description', 'Working type', 'Contact', 'Achievements', 'Base salary', 'Equity']
total_values = [[] for _ in total_columns]

for single_job in job_title_list:
    count += 1
    print("*****Processing job {job}*****".format(job=single_job))
    print("Type of position: " + type_list[single_job])
    print("Job location: " + location_list[single_job])
    print("Job description: " + description_list[single_job])
    print("Working type: " + (",".join(str(x) for x in industry_list[single_job])))
    print("Contract: " + contract_list[single_job])
    print("Achievements: " + achievements_list[single_job])
    print("Base salary: " + salary_list[single_job])
    print("Equity: " + equity_list[single_job])
    
    print("label: " + (",".join(str(x) for x in label_list[single_job])))
    
    single_values = [
        single_job,
        type_list[single_job],
        location_list[single_job],
        description_list[single_job],
        ",".join(str(x) for x in industry_list[single_job]),
        contract_list[single_job],
        achievements_list[single_job],
        salary_list[single_job],
        equity_list[single_job]
    ]
    for value_index in range(len(total_columns)):
        total_values[value_index].append(single_values[value_index])
final_result = {}
for column_index in range(len(total_columns)):
    final_result.update({
        total_columns[column_index]: total_values[column_index]
    })
final_result = PD.DataFrame(final_result)

print("*****Total count {count}*****".format(count=count))
file_name = "jobs.csv"
print("*****Save the result to file {file}*****".format(file=file_name))
final_result.to_csv(file_name)


*****Processing job Parity + Senior Team Assistant in Org Strategy*****
Type of position: Full-time
Job location: Remote (Central European Time (CET))
Job description: 
Parity is one of the world's most experienced core blockchain infrastructure companies, having built and pioneered some of the most advanced technologies in the blockchain sector. Parity was founded by Dr. Gavin Wood, co-founder and former CTO of Ethereum, the primary engineer behind the Ethereum Virtual Machine (EVM), inventor of the Solidity programming language, and primary author of the Ethereum Yellow paper.
Based in Berlin, London, and Lisbon with over 340 employees worldwide, Parity has built clients for Ethereum, Bitcoin, and Zcash and has pioneered a completely new, next-generation blockchain protocol with Polkadot and the framework its built with, Substrate. Parity builds the open-source technologies needed to power an unstoppable, decentralised webknown as Web3and helps developers and organisations implement 

In [ ]:
# run here

# 采用静态捕获的方法(已优化)

In [ ]:
# make the bs soup into a list
with open('jobs.html', 'r', encoding='utf-8') as f:
 	soup_list = BeautifulSoup(f.read(), 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")

In [ ]:
# get the initializd infomation
tag_list = []

job_title_list = []
type_list = {}
industry_list = {}
contract_list = {}
location_list = {}
equity_list = {}
achievements_list = {}
# loop = 0

for tag in soup_list.find_all('div',class_="job-list-job"):
        # print(tag)
        job_title_list.append(tag.find('a', target = "_blank").text)
        type_list[tag.find('a', target = "_blank").text] = 'Full-time'
        industry_list[tag.find('a', target = "_blank").text] = ['Others']
        contract_list[tag.find('a', target = "_blank").text] = 'talentre'
        # location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-badge-text").text if (not tag.find('span', class_ = "job-list-badge-text").text[0:6] == "Posted") else "Not Posted"
        location_list[tag.find('a', target = "_blank").text] = tag.find('span', class_ = "job-list-company-meta-item").text
        equity_list[tag.find('a', target = "_blank").text] = "Not Posted"
        achievements_list
        # loop += 1
        # if(loop == 10):
        #     break
        # for tag_ in tag.find_all('span',attrs={ 'data-toggle': 'tooltip'}):
        #     if('$' in tag_.text):
        #         tmp = tag_.text.split('$')[1].replace(',','').replace(')','')


In [ ]:
# adjust the type_list
with open('jobs_internal.html', 'r', encoding='utf-8') as f:
 	soup_list_internal = BeautifulSoup(f.read(), 'lxml').body.find('div', id="root-react-container").find("div", class_ = "boards-body fenbushi-capital").find("div", class_ = "job-list")

In [ ]:
for tag in soup_list_internal.find_all('div',class_="job-list-job"):
    type_list[tag.find('a', target = "_blank").text] = 'Internship'

In [ ]:
# adjust the industry_list
industry = ['Engineer', 'Marketing']

In [ ]:
# show all the results
count = 0
df = PD.DataFrame(columns=['Job title', 'Type of position', 'Job Location', 'Job description', 'Working type', 'Contact', 'Achievements', 'Base salary', 'Equity'])
for single_job in job_title_list:
    count += 1
    print("*****Processing job {job}*****".format(job=single_job))
    print("Type of position: " + type_list[single_job])
    print("Job location: " + location_list[single_job])
    print("Working type: " + (",".join(str(x) for x in industry_list[single_job])))
    print("Contract: " + contract_list[single_job])
    print("Achievements: " + achievements_list[single_job])
    print("Equity: " + equity_list[single_job])
    print("Achievements: " + achievements_list[single_job])
print("*****Total count {count}*****".format(count=count))

*****Processing job Business Operations Intern*****
Type of position: Internship
Job location: New York City Metropolitan Area, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Digital Marketing Assistant (Intern)*****
Type of position: Internship
Job location: New York, NY, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Marketing Assistant (Intern)*****
Type of position: Internship
Job location: New York, NY, US
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Platform Engineering Intern*****
Type of position: Internship
Job location: New York, NY  / Seattle, WA / San Francisco Bay Area, CA / Remote
Working type: Others
Contract: talentre
Equity: Not Posted
*****Processing job Internship Spring/Summer 2023 - Software Engineer, Tools*****
Type of position: Internship
Job location: New York, NY  / Seattle, WA / San Francisco Bay Area, CA / Remote
Working type: Others
Contract: talentre
Equi